In [1]:
!pip install face_recognition

  Using cached face_recognition-1.3.0-py2.py3-none-any.whl (15 kB)
  Using cached face_recognition_models-0.3.0.tar.gz (100.1 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached dlib-19.24.2.tar.gz (11.8 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for dlib: filename=dlib-19.24.2-cp38-cp38-win_amd64.whl size=2835401 sha256=01320ee938810384bfecf789d8a55d704bc82b967c391aabdcc5b2da32703027
  Stored in directory: c:\users\hp 2021\appdata\local\pip\cache\wheels\21\03\71\98fd66c954809f33662e781bd02c4363cced2fa1c76178f812


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566173 sha256=f4ffac1cfa9edea97ca919fbad9f948a104795475522a0d6b1492581b149fba0
  Stored in directory: c:\users\hp 2021\appdata\local\pip\cache\wheels\b4\4b\8f\751e99d45f089bdf366a7d3e5066db3c2b84a62e4377f534d7
Successfully built dlib face-recognition-models


In [1]:
import cv2
import numpy as np
import pandas as pd
import face_recognition
import os
from datetime import datetime

In [12]:
path = 'dataset'
images = []
classNames = []
myList = os.listdir(path)
print(myList)

['bill.jpg', 'elon.jpg', 'steve.jpg', 'zaki.jpg']


In [13]:
for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])
print(classNames)

['bill', 'elon', 'steve', 'zaki']


In [14]:
def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList

In [15]:
def markAttendance(name):
    with open('Attendance.csv','r+') as f:
        myDataList = f.readlines()
        nameList = []
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
        if name not in nameList:
            now = datetime.now()
            dtString = now.strftime('%H:%M:%S')
            f.writelines(f'\n{name},{dtString}')
        

encodeListKnown = findEncodings(images)
print('Encoding Complete')


Encoding Complete


In [12]:

cap = cv2.VideoCapture(0)
frame_resizing = 0.25

while True:
    success, img = cap.read()
    imgS = cv2.resize(img, (0, 0), fx=frame_resizing, fy=frame_resizing)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
 
    facesCurFrame = face_recognition.face_locations(imgS)
    encodesCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)
 
    for encodeFace, faceLoc in zip(encodesCurFrame, facesCurFrame):
        matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
        print('matches', matches)
        faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
        print(faceDis)
        matchIndex = np.argmin(faceDis)
        if matches[matchIndex]:
            name = classNames[matchIndex].upper()
            print(name)
            
            faceLoc = np.array(faceLoc)
            faceLoc = faceLoc / 0.25
            faceLoc = faceLoc.astype(int)
            y1, x2, y2, x1 = faceLoc[0], faceLoc[1], faceLoc[2], faceLoc[3]
         
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.rectangle(img, (x1, y2-35), (x2, y2), (0, 255, 0), cv2.FILLED)
            cv2.putText(img, name, (x1+6, y2-6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
            markAttendance(name)
            
        else:
            name = "not found"
            print(name)
            
            faceLoc = np.array(faceLoc)
            faceLoc = faceLoc / 0.25
            faceLoc = faceLoc.astype(int)
            y1, x2, y2, x1 = faceLoc[0], faceLoc[1], faceLoc[2], faceLoc[3]
         
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
            cv2.rectangle(img, (x1, y2-35), (x2, y2), (0, 0, 255), cv2.FILLED)
            cv2.putText(img, name, (x1+6, y2-6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
            markAttendance(name)
 
    cv2.imshow('Webcam', img)
    if cv2.waitKey(16) & 0xff == ord("x"):
        break
        
cap.release()
cv2.destroyAllWindows()

matches [False, False, False, True]
[0.81781216 0.73664351 0.84162651 0.54328239]
ZAKI
matches [False, False, False, True]
[0.82330708 0.76997508 0.84314425 0.49332017]
ZAKI
matches [False, False, False, True]
[0.8103902  0.77072623 0.82711758 0.48302702]
ZAKI
matches [False, False, False, True]
[0.83290906 0.76780665 0.84741812 0.48954742]
ZAKI
matches [False, False, False, True]
[0.82809241 0.74338822 0.84270066 0.48722207]
ZAKI
matches [False, False, False, True]
[0.83926583 0.76143456 0.83864274 0.50645831]
ZAKI
matches [False, False, False, True]
[0.7904235  0.75879828 0.82852369 0.5110851 ]
ZAKI
matches [False, False, False, True]
[0.80600378 0.76359634 0.81370477 0.47691021]
ZAKI
matches [False, False, False, True]
[0.76385994 0.74281244 0.76266691 0.49488385]
ZAKI
matches [False, False, False, True]
[0.78456215 0.70364389 0.78395695 0.44785686]
ZAKI
matches [False, False, False, True]
[0.77160665 0.7305216  0.77723192 0.47821597]
ZAKI
matches [False, False, False, True]
[0.7973

matches [False, False, False, True]
[0.83057471 0.75721175 0.81984594 0.48113175]
ZAKI
matches [False, False, False, True]
[0.81428705 0.7603481  0.7897205  0.53712334]
ZAKI
matches [False, False, False, True]
[0.81192713 0.75786798 0.79501232 0.49132237]
ZAKI
matches [False, False, False, True]
[0.82253178 0.77522455 0.80495648 0.49419152]
ZAKI
matches [False, False, False, True]
[0.8289924  0.75960327 0.79961537 0.52594322]
ZAKI
matches [False, False, False, True]
[0.82381618 0.75716811 0.79386488 0.50863096]
ZAKI
matches [False, False, False, True]
[0.80505053 0.77268648 0.77399904 0.52930454]
ZAKI
matches [False, False, False, True]
[0.83316558 0.77048098 0.77790887 0.54580464]
ZAKI
matches [False, False, False, True]
[0.81694788 0.7825972  0.7822703  0.50340108]
ZAKI
matches [False, False, False, True]
[0.81624664 0.76915145 0.79095129 0.50566131]
ZAKI
matches [False, False, False, True]
[0.83072386 0.76191995 0.78935523 0.51094801]
ZAKI
matches [False, False, False, True]
[0.8354

In [ ]:
#Yes, the model used in the provided code is the Histogram of Oriented Gradients (HOG) model for face detection. HOG is a popular feature descriptor that captures the shape and appearance of objects in an image. It's commonly used in object detection tasks, including face detection. In the context of face recognition, the HOG model helps in detecting faces within an image or video frame.
#عندما تُسألين عن اسم النموذج في السياق الذي ذكرتِه، يمكنكِ الرد بأن النموذج المستخدم هو نموذج Histogram of Oriented Gradients (HOG) للكشف عن الوجوه.
#نعم، النموذج المستخدم في الكود المقدم هو نموذج Histogram of Oriented Gradients (HOG) لكشف الوجه. HOG هو واصف ميزة شائع يلتقط شكل ومظهر الكائنات في الصورة. يشيع استخدامه في مهام اكتشاف الأجسام، بما في ذلك اكتشاف الوجه. في سياق التعرف على الوجوه، يساعد نموذج HOG في اكتشاف الوجوه داخل صورة أو إطار فيديو.